# Implement CBC mode
CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.

In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.

The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.

Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.

The file here is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)

### Don't cheat.
Do not use OpenSSL's CBC code to do CBC mode, even to verify your results. What's the point of even doing this stuff if you aren't going to learn from it?

In [1]:
from cryptopals import *

In [2]:
def encrypt_AES_128_ECB(plaintext, key):
    BLOCK_SIZE = 16
    '''Encrypts a plaintext string using a given key, using AES-128 in ECB mode.'''
    if len(key) != BLOCK_SIZE:
        raise ValueError('Key for AES-128 must be %d bytes' % BLOCK_SIZE)
    
    padded_plaintext = str(pad_pkcs7(plaintext, BLOCK_SIZE))
    return AES.new(key, AES.MODE_ECB).encrypt(padded_plaintext)


def encrypt_AES_128_ECB_base64(plaintext_base64, key):
    return encrypt_AES_128_ECB(str(base64_to_bytes(plaintext_base64)), key)


def encrypt_AES_128_CBC(plaintext, key, IV):
    BLOCK_SIZE = 16
    '''Encrypts a plaintext string using a given key (str) and IV (str), using AES-128 in CBC mode.'''
    if len(key) != BLOCK_SIZE:
        raise ValueError('Key for AES-128 must be %d bytes' % BLOCK_SIZE)
        
    if len(IV) != BLOCK_SIZE:
        raise ValueError('IV for AES-128 in CBC mode must be %d bytes' % BLOCK_SIZE)
    
    AES_128_ECB = AES.new(key, AES.MODE_ECB)
    padded_plaintext = pad_pkcs7(plaintext, BLOCK_SIZE)
    
    ciphertext = b''
    for start_idx in xrange(0, len(padded_plaintext), BLOCK_SIZE):
        block_cipher_input = fixed_xor(bytearray(IV), padded_plaintext[start_idx : start_idx+BLOCK_SIZE])
        block_cipher_output = AES_128_ECB.encrypt(str(block_cipher_input))
        ciphertext += block_cipher_output
        IV = block_cipher_output
    
    return ciphertext


def decrypt_AES_128_CBC(ciphertext, key, IV):
    BLOCK_SIZE = 16
    '''Decrypts a ciphertext string using a given key (str) and IV (str), using AES-128 in CBC mode.'''
    if len(key) != BLOCK_SIZE:
        raise ValueError('Key for AES-128 must be %d bytes' % BLOCK_SIZE)
        
    if len(IV) != BLOCK_SIZE:
        raise ValueError('IV for AES-128 in CBC mode must be %d bytes' % BLOCK_SIZE)
    
    if len(ciphertext) % BLOCK_SIZE != 0:
        raise ValueError('Ciphertext for AES-128 in CBC mode must be a multiple of %d bytes' % BLOCK_SIZE)
        
    AES_128_ECB = AES.new(key, AES.MODE_ECB)
    
    plaintext = b''
    for start_idx in xrange(0, len(ciphertext), BLOCK_SIZE):
        block_cipher_input = ciphertext[start_idx : start_idx+BLOCK_SIZE]
        block_cipher_output = AES_128_ECB.decrypt(str(block_cipher_input))
        plaintext += fixed_xor(bytearray(IV), bytearray(block_cipher_output))
        IV = block_cipher_input
    
    return plaintext

In [3]:
IV = hex_to_bytes('0'*32)
IV

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [4]:
ct = encrypt_AES_128_CBC("Hello, world! It's CryptoPals!!", "YELLOW SUBMARINE", IV)
ct

'!\xb2\xfb\xb0w\xfa;Yw\x03h\xc7\xa2\x16/\xe1\xa11M\xc91\xb9\x0eaA\x9a\xcc\xd1\xacr;J'

In [5]:
decrypt_AES_128_CBC(ct, "YELLOW SUBMARINE", IV)

bytearray(b"Hello, world! It\'s CryptoPals!!\x01")

In [6]:
ct = ''
with open('Data/10.txt') as f:
    for row in f:
        ct += row.strip()

print decrypt_AES_128_CBC(base64_to_bytes(ct), "YELLOW SUBMARINE", IV)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 
